#### Cleaning Steps - Immigration Data

  - Transform all Integer Variables.
  - Transform dates in different formats to datetime.
  - For variables like `i94cit`, `i94res` they can be an additional Dimension Table.

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from numpy import nan
import numbers
from os import path 
from glob import glob
import re

sas_data_parquet = glob('./sas-data/*.parquet')
immigration_data_full = glob('./immigration-data/18-83510-I94-Data-2016/*.sas7bdat')
immigration_data_sample = './immigration_data_sample.csv'

def is_leap(year):
    return (( year%400 == 0)or (( year%4 == 0 ) and ( year%100 != 0)))

def parse_quarter(quarter_date):
    regex = 'Q(\d{1})(\d{2})(\d{4})'
    results = re.search(regex, quarter_date)
    quarter, day, year = [int(n) for n in results.groups()]
    quarter_start = {
        1: 1,
        2: 4,
        3: 7,
        4: 10  
    } 
    date = datetime(year, quarter_start[quarter], 1)
    delta = timedelta(days=day)
    return date + delta       

def parser(days): 
    if pd.isnull(days):
        return None
    if isinstance(days, numbers.Number):
        return datetime(1960, 1, 1) + timedelta(int(float(days)))
    return None

def parser_dtadfile(date_string): 
    try:
        if pd.isnull(date_string):
            return None
        if not isinstance(date_string, str):
            parts = str(date_string).split(".")[0]
            return datetime.strptime(parts, "%Y%m%d")
        if date_string.startswith("Q"):
            return parse_quarter(str(date_string))
        return datetime.strptime(date_string, "%Y%m%d")
    except ValueError as e:
        print(f"ValueError: ERROR {date_string}")
        return None
        

def parser_dtaddto(date_string): 
    try:
        if pd.isnull(date_string) or date_string == "D/S":
            return None
        if not isinstance(date_string, str):
            parts = str(date_string).split(".")[0]
            return datetime.strptime(parts, "%m%d%Y")
        if date_string.startswith("Q"):
            return parse_quarter(str(date_string))
        return datetime.strptime(date_string, "%m%d%Y")
    except ValueError as e:
        print(f"ValueError: ERROR {date_string}")
        return None

immigration_cols = [
    'cicid', 'i94yr', 'i94mon', 
    'i94cit', 'i94res', 'i94port',
    'arrdate', 'i94mode', 'i94addr',
    'depdate', 'i94bir', 'i94visa', 
    'count', 'dtadfile', 'visapost', 
    'occup', 'entdepa', 'entdepd', 
    'entdepu', 'matflag', 'biryear', 
    'dtaddto', 'gender', 'insnum', 
    'airline', 'admnum', 'fltno', 
    'visatype'
]

integer_columns = ['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94mode', 'i94bir', 'i94visa', 'count', 'biryear', 'gender', 'insnum', 'admnum']

print(parse_quarter('Q4142016'))

2016-10-15 00:00:00


In [2]:
# Parsing SAS Files
# The whole file is ~500mb and processing locally might take a long time
CHUNK_SIZE=5
for chunk in pd.read_sas(immigration_data_full[0], format='sas7bdat', encoding="ISO-8859-1", chunksize=CHUNK_SIZE):
    chunk[integer_columns] = chunk[integer_columns].apply(pd.to_numeric, errors='coerce', downcast='integer')
    chunk['arrdate'] = chunk['arrdate'].apply(parser)
    chunk['depdate'] = chunk['depdate'].apply(parser)
    chunk['dtadfile'] = chunk['dtadfile'].apply(parser_dtadfile)
    chunk['dtaddto'] = chunk['dtaddto'].apply(parser_dtaddto)
    display(chunk.iloc[:2]) # chunk is size CHUNK_SIZE
    break
    

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,46,2016,12,129,129,HOU,2016-12-01,1,TX,2016-12-14,...,NaN,M,1970,2018-05-26,NaN,NaN,RS,97554138030,7715,E2
1,56,2016,12,245,245,NEW,2016-12-01,1,OH,2017-01-16,...,NaN,M,1988,NaT,NaN,NaN,CA,90623718230,819,F1


In [3]:
# Cleaning Parquet File
immigration_data_df_from_parquet = pd.read_parquet(sas_data_parquet[0])
immigration_data_df_from_parquet[integer_columns] = immigration_data_df_from_parquet[integer_columns].apply(pd.to_numeric, errors='coerce', downcast='integer')
immigration_data_df_from_parquet['arrdate'] = immigration_data_df_from_parquet['arrdate'].apply(parser)
immigration_data_df_from_parquet['depdate'] = immigration_data_df_from_parquet['depdate'].apply(parser)
immigration_data_df_from_parquet['dtadfile'] = immigration_data_df_from_parquet['dtadfile'].apply(parser_dtadfile)
immigration_data_df_from_parquet['dtaddto'] = immigration_data_df_from_parquet['dtaddto'].apply(parser_dtaddto)

display(immigration_data_df_from_parquet.iloc[:1])

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,459651,2016,4,135,135,ATL,2016-04-03,1,FL,2016-04-15,...,None,M,1962,2016-07-01,NaN,NaN,VS,55556253633,00115,WT


In [4]:
# Cleaning CSV
immigration_data_df_from_sample = pd.read_csv(immigration_data_sample,
                                              parse_dates=[7, 10],
                                              date_parser=parser,
                                              header=0)

immigration_data_df_from_sample[integer_columns] = immigration_data_df_from_sample[integer_columns].apply(pd.to_numeric, errors='coerce', downcast='integer')

immigration_data_df_from_sample['dtadfile'] = immigration_data_df_from_sample['dtadfile'].apply(parser_dtadfile)
immigration_data_df_from_sample['dtaddto'] = immigration_data_df_from_sample['dtaddto'].apply(parser_dtaddto)
immigration_data_df_from_sample.iloc[:1]


,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316,2016,4,209,209,HHW,NaT,1,HI,...,NaN,M,1955,2016-07-20,NaN,NaN,JL,56582674633,00782,WT


In [2]:
from os import getcwd
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from immigration_lib.aws_tools import create_spark_session

aws_config = f"{getcwd()}/aws_config.cfg"
spark_extra_conf = {"spark.jars.packages": "saurfang:spark-sas7bdat:2.0.0-s_2.11"}
spark = create_spark_session(aws_config, spark_extra_conf, "Cleaning I94 Records")

In [6]:
sas_spark = spark.read.format('com.github.saurfang.sas.spark') \
            .load(immigration_data_full[0])

sas_spark.printSchema()

display(sas_spark.limit(5).toPandas())

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,46.0,2016.0,12.0,129.0,129.0,HOU,20789.0,1.0,TX,20802.0,...,None,M,1970.0,05262018,M,None,RS,9.755414e+10,7715,E2
1,56.0,2016.0,12.0,245.0,245.0,NEW,20789.0,1.0,OH,20835.0,...,None,M,1988.0,D/S,F,None,CA,9.062372e+10,819,F1
2,67.0,2016.0,12.0,512.0,512.0,PEV,20789.0,2.0,MD,20794.0,...,None,M,1968.0,06012017,M,5920,None,8.010503e+10,None,B2
3,68.0,2016.0,12.0,512.0,512.0,PEV,20789.0,2.0,FL,20792.0,...,None,M,1970.0,06012017,F,5920,None,8.010511e+10,None,B2
4,69.0,2016.0,12.0,512.0,512.0,PEV,20789.0,2.0,HI,20792.0,...,None,M,1968.0,06012017,M,5920,None,8.010511e+10,None,B2


In [8]:
occupations = sas_spark.groupBy('occup').agg({"occup":"count"}).toPandas()

print("""There are
         - {} None values in occupation,
         - {} different types of occupations
         - All not null occupations account for {} values""".format(
            sas_spark.select('occup')
                .withColumn('isNull_occup', col('occup').isNull())
                .where('isNull_occup = True')
                .count(), 
            occupations.shape[0], 
            occupations['count(occup)'].sum()
        )
     )

There are
         - 3423479 None values in occupation,
         - 107 different types of occupations
         - All not null occupations account for 9511 values


In [9]:
occupations_and_visas = sas_spark.groupBy('visatype', 'occup').count().toPandas()

In [10]:
from immigration_lib.visatypes import non_immigrant_visas, immigrant_visas

visa_types = occupations_and_visas.visatype.unique().tolist()

print("Visa Types found with an Occupation: {}".format(",".join(visa_types)))
for visatype in visa_types:
    if visatype in non_immigrant_visas:
        print("nonimmigrant", visatype, non_immigrant_visas[visatype])
    elif visatype in immigrant_visas:
        print("immigrant", visatype, immigrant_visas[visatype])
    else:
        print("No valid visa found for {}, is it a data entry error?".format(visatype))
        


Visa Types found with an Occupation: M1,F1,E2,F2,I,B2,B1,GMT,E1,M2,GMB,WT,I1,CPL,CP,WB,SBP
nonimmigrant M1 Vocational Student or Other Nonacademic Student
nonimmigrant F1 Student in an academic or language training program
nonimmigrant E2 Treaty Investor, Spouse or Child
nonimmigrant F2 Spouse or Child of F1
nonimmigrant I Representative of Foreign Information Media, Spouse and Child
nonimmigrant B2 Temporary Visitor for Pleasure
nonimmigrant B1 Temporary Visitor for Business
No valid visa found for GMT, is it a data entry error?
nonimmigrant E1 Treaty Trader, Spouse or Child
nonimmigrant M2 Spouse or Child of M1
No valid visa found for GMB, is it a data entry error?
No valid visa found for WT, is it a data entry error?
No valid visa found for I1, is it a data entry error?
No valid visa found for CPL, is it a data entry error?
No valid visa found for CP, is it a data entry error?
No valid visa found for WB, is it a data entry error?
No valid visa found for SBP, is it a data entry error

In [11]:
sas_file_total = sas_spark.count()

print(f"Total rows in 1 SAS file {sas_file_total}")
none_values = sas_spark \
                .select([count(when(col(c).isNull(), c)).alias(c) for c in sas_spark.columns]) \
                .toPandas()
none_values = none_values.rename(index={0: 'count'}).T.sort_values("count",ascending=False)
none_values['count_percentage'] = ((none_values['count']/sas_file_total) * 100).astype(int)

Total rows in 1 SAS file 3432990


In [12]:
display(none_values[none_values['count_percentage'] > 0])

,count,count_percentage
entdepu,3431429,99
occup,3423479,99
insnum,2502515,72
visapost,1784575,51
depdate,214788,6
entdepd,203852,5
matflag,203852,5
i94addr,166825,4
airline,93875,2
gender,67606,1


#### Extra Dimensions to Add

In [13]:
from immigration_lib.i94labels import i94cntyl, i94prtl, i94addrl, i94mode
# i94cit i94res i94port i94addr i94mode

# sas_spark.i94cit
# sas_spark.i94res
# sas_spark.i94port
# sas_spark.i94addr
# sas_spark.i94mode


#### Conclusions for Cleaning Immigration Data

1. Based on the low number of rows with occupations, and the high amount of missing data when finding visa types with occupations, we remove it from our staging table.
2. Remove other columns with over 90% of Null values. (`entdepu`)
2. Remove `count` because it is an internal variable.
3. Remove `insnum` because there is no source to join it with.

## Single ELT cell to load from SAS to S3

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numbers
import re
from os import getcwd
from glob import glob
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from immigration_lib.aws_tools import create_spark_session

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

def is_leap(year):
    return (( year%400 == 0)or (( year%4 == 0 ) and ( year%100 != 0)))

def parse_quarter(quarter_date):
    regex = 'Q(\d{1})(\d{2})(\d{4})'
    results = re.search(regex, quarter_date)
    quarter, day, year = [int(n) for n in results.groups()]
    quarter_start = {
        1: 1,
        2: 4,
        3: 7,
        4: 10  
    } 
    date = datetime(year, quarter_start[quarter], 1)
    delta = timedelta(days=day)
    return date + delta       

def parser(days): 
    if pd.isnull(days):
        return None
    if isinstance(days, numbers.Number):
        return datetime(1960, 1, 1) + timedelta(int(float(days)))
    return None

def parser_dtadfile(date_string): 
    try:
        if pd.isnull(date_string):
            return None
        if not isinstance(date_string, str):
            parts = str(date_string).split(".")[0]
            return datetime.strptime(parts, "%Y%m%d")
        if date_string.startswith("Q"):
            return parse_quarter(str(date_string))
        return datetime.strptime(date_string, "%Y%m%d")
    except ValueError as e:
        if len(date_string) > 8:
            print(f"ValueError: ERROR {date_string}")
        return None
        

def parser_dtaddto(date_string): 
    try:
        if pd.isnull(date_string) or date_string == "D/S":
            return None
        if not isinstance(date_string, str):
            parts = str(date_string).split(".")[0]
            return datetime.strptime(parts, "%m%d%Y")
        if date_string.startswith("Q"):
            return parse_quarter(str(date_string))
        return datetime.strptime(date_string, "%m%d%Y")
    except ValueError as e:
        if len(date_string) > 8:
            print(f"ValueError: ERROR {date_string}")
        return None

integer_columns = ['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94mode', 'i94bir', 'i94visa', 'count', 'biryear', 'insnum', 'admnum']

def process_sas_file(raw_sas_df):
    sas_spark_with_int = raw_sas_df
    for integer_column in integer_columns:
        sas_spark_with_int = sas_spark_with_int \
                        .withColumn(integer_column, col(integer_column).cast("int"))

    date_udf = udf(parser, DateType())
    date_dtadfile_udf = udf(parser_dtadfile, DateType())
    date_dtaddto_udf = udf(parser_dtaddto, DateType())
    filtered_sas_spark = sas_spark_with_int \
                    .withColumn("arrdate", date_udf(col("arrdate"))) \
                    .withColumn("depdate", date_udf(col("depdate"))) \
                    .withColumn("dtadfile", date_dtadfile_udf(col("dtadfile"))) \
                    .withColumn("dtaddto", date_dtaddto_udf(col("dtaddto"))) \
                    .drop('count', 'entdepu', 'occup', 'insnum') \
                    .dropDuplicates()
    return filtered_sas_spark

output_data = "s3a://claudiordgz-udacity-dend"
immigration_data_files = glob('./immigration-data/18-83510-I94-Data-2016/*.sas7bdat')
aws_config = f"{getcwd()}/aws_config.cfg"
spark = create_spark_session(aws_config, {"spark.jars.packages": "saurfang:spark-sas7bdat:2.0.0-s_2.11"}, "Moving I94 Records to S3")

for i, immigration_data in enumerate(immigration_data_files):
    sas_spark = spark.read.format('com.github.saurfang.sas.spark').load(immigration_data)
    sas_df = process_sas_file(sas_spark)
    if i == 0:
        sas_df.printSchema()
        display(sas_df.limit(1).toPandas())
    print(f'Processsing: {i}, {immigration_data}')
    sas_df.write.parquet(f'{output_data}/capstone/staging_i94_records', mode='append', partitionBy=['i94yr', 'i94mon'])

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: date (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: date (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- dtadfile: date (nullable = true)
 |-- visapost: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: integer (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,dtadfile,visapost,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
0,1587,2016,12,438,438,CLT,2016-12-01,1,MA,2016-12-05,68,2,2016-12-01,None,H,O,M,1948,2017-02-13,F,AA,2147483647,822,WT


Processsing: 0, ./immigration-data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat
Processsing: 1, ./immigration-data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
Processsing: 2, ./immigration-data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
Processsing: 3, ./immigration-data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
Processsing: 4, ./immigration-data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
Processsing: 5, ./immigration-data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
Processsing: 6, ./immigration-data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
Processsing: 7, ./immigration-data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
Processsing: 8, ./immigration-data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
Processsing: 9, ./immigration-data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
Processsing: 10, ./immigration-data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
Processsing: 11, ./immigration-data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
